In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import networkx as nx
import numpy as np
from lib.graph_local_classes import GraphStructure, GraphParams, InnerGraphSimulation

In [3]:

from lib.graph_enumerator import generate_graphs
from lib.node_semantics import Node_Name_Rule, Edge_Semantics_Rule
from lib import config, result_config
from lib.likelihood_calculations_shared_params import Inference
from lib.utils import filename_utility
from lib.misc import cond_to_data
import time

In [8]:
graph_iter = generate_graphs(**config.generator_dictionary)
graphs = list(graph_iter)
for graph in graphs:
    Node_Name_Rule.graph_semantics_apply(graph,config.node_semantics)
    Edge_Semantics_Rule.graph_semantics_apply(graph,config.edge_semantics)

inference_obj = Inference()

options = config.options
options['data_sets']=cond_to_data(config.conds[1,:])



In [9]:
result_graphs, result_posterior, result_loglik, result_dict = inference_obj.p_graph_given_d(graphs,config.options)


In [13]:
small_val = .05

cond1 = [0,small_val,small_val,small_val]
cond2 = [0,1,3,2]
cond3 = [0,3,2,1]
cond4 = [0,1,2,2]

conds = np.array([cond1,cond2,cond3,cond4])

array([ 0.05,  1.  ,  3.  ,  1.  ])

In [6]:
graph_iter = generate_graphs(**config.generator_dictionary)
graphs = list(graph_iter)
for graph in graphs:
    Node_Name_Rule.graph_semantics_apply(graph,config.node_semantics)
    Edge_Semantics_Rule.graph_semantics_apply(graph,config.edge_semantics)

max_graph = graphs[0]
num_graphs = len(graphs)
num_params = config.options["param_sample_size"]
loglikelihood_by_param = np.zeros(shape = (num_params,num_graphs))
max_graph_params = GraphParams.from_networkx(max_graph) # fix this when you can
max_graph_params.sample()
# blah = inference_obj.subgraph_loglik(graphs[1],max_graph_params,options=config.options)


{'lambda0': [0.0424809387754264],
 'mu': array([  2.71933940e+00,   4.53054052e+00,   5.89509979e+00,
          1.54304435e-03,   5.21191696e+00,   4.71668377e-01,
          6.78399058e+00,   3.41861445e-01,   1.64180234e+01,
          4.34800210e-02,   2.87475581e+00,   1.28164203e+01,
          1.99377316e-01,   1.65249193e+00,   2.32301474e+00]),
 'n': 15,
 'names': [('A_int', 'A_obs'),
  ('A_int', 'A_★'),
  ('A_★', 'A_obs'),
  ('A_★', 'B_★'),
  ('A_★', 'C_★'),
  ('A_★', 'D_★'),
  ('B_★', 'B_obs'),
  ('B_★', 'C_★'),
  ('B_★', 'D_★'),
  ('C_★', 'B_★'),
  ('C_★', 'C_obs'),
  ('C_★', 'D_★'),
  ('D_★', 'B_★'),
  ('D_★', 'C_★'),
  ('D_★', 'D_obs')],
 'p': 0.8,
 'psi': array([  1.55579701e-01,   2.88112347e-02,   6.73583746e-02,
          1.12340298e-04,   4.19068282e-02,   5.73180822e-02,
          1.33962671e-02,   4.27577446e-02,   3.51847135e-02,
          2.30080104e-03,   4.48371358e-02,   1.03234279e-01,
          2.27167638e-02,   2.44328657e-02,   1.02862783e-01]),
 'psi_shape': 

In [26]:
class ContObj:
    
    def __iter__(self):
        return iter(['hi', 'banana','red'])
    
    def __getitem__(self, value):
        
        return np.ones(shape=[value]*value)

In [29]:
cont_obj = ContObj()

In [33]:
cont_obj[4][:, :,3, :]

array([[[ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.]],

       [[ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.]],

       [[ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.]],

       [[ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.]]])

In [25]:
for x in iter(cont_obj):
    print(x)

TypeError: 'ConfObj' object is not iterable

In [8]:
result_graphs